In [1]:
#Feature selection for implementing Dr. Elnitski's algorithm
import os
import csv
import statistics

inpdat = "../../coadreadDAThugocombine.csv"

with open(inpdat,'r') as f:
    it = csv.reader(f)
    listit = list(it)
    
print("col # of old:",len(listit[0]))
print("row #:",len(listit))

col # of old: 15949
row #: 225


In [2]:
endcol = len(listit[0])-1
endrow = len(listit)

#count feats with 0 cimp-
posscols = [] #only get columns w at least 2 CIMP+ coverage
count = count1 = count2 = 0
maxFP = 0
for y in range(1,endcol):
    FPcount = 0
    for x in range(1,endrow):
        if(float(listit[x][y]) == 1.0):
            if(float(listit[x][endcol]) == -1.0 or float(listit[x][endcol]) == 2.0):
                FPcount += 1
    if(FPcount == 0):
        count += 1
    if(FPcount <= 1):
        count1 += 1
    if(FPcount <= 2):
        count2 += 1
    if(FPcount > maxFP):
        maxFP = FPcount

print("max CIMP-",maxFP)        
print("num feats with 0 CIMP-: " + str(count))
print("num feats with 1 or less CIMP-: " + str(count1))
print("num feats with 2 or less CIMP-: " + str(count2))

newdatset=[
    ["Number CIMP-","Number CIMP+","Total # of Features","TP","FP","TN","FN"]
]

def calcitout(fp,tp,newdatset):
    posscount = []
    for y in range(1,endcol):
        TPcount = 0
        FPcount = 0
        for x in range(1,endrow):
            if(float(listit[x][y]) == 1.0):
                if(float(listit[x][endcol]) == 1.0):
                    TPcount += 1
                else:
                    FPcount += 1
        if(TPcount >= tp):
            if(FPcount <= fp):
                posscount.append(listit[0][y])
    newdatset.append([fp,tp,len(posscount)])
    return posscount

#looking at all features overall
def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])-1
    results = {}
    for x in range(0, numrows):
        for y in range(0, numcols): #program won't be able to see background/foreground
            if (testdata[x][y] == "1.0"):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = "1.0"
    for x in range(0,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = "-1.0"
    return results
def evalResults(dictofseqs,givendat,i):
    falsepos = falseneg = truepos = trueneg = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(dictofseqs[givendat[x][0]] != givendat[x][len(givendat[0])-1]):
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    falsepos += 1
                else:
                    falseneg += 1
            else:
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    truepos += 1
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    mytup = (truepos,falsepos,trueneg,falseneg)
    return mytup

max CIMP- 154
num feats with 0 CIMP-: 1140
num feats with 1 or less CIMP-: 5073
num feats with 2 or less CIMP-: 8439


In [3]:
count = 1 #used to hold newdatset index to write to
for x in range(20,-1,-1): #for cimp- allowance 20 to 0...
    #now CIMP+ for 3 to 20...
    for y in range(3,21):
        posscount = calcitout(x,y,newdatset)
        newdatset[count].extend(evalResults(runGreedyOnTest(posscount,listit),listit,1)) #attach TP, FP, TN, FN
        count += 1
newdatset

[['Number CIMP-',
  'Number CIMP+',
  'Total # of Features',
  'TP',
  'FP',
  'TN',
  'FN'],
 [20, 3, 2165, 32, 192, 0, 0],
 [20, 4, 1042, 32, 192, 0, 0],
 [20, 5, 498, 32, 192, 0, 0],
 [20, 6, 253, 32, 190, 2, 0],
 [20, 7, 123, 32, 180, 12, 0],
 [20, 8, 58, 30, 155, 37, 2],
 [20, 9, 30, 29, 110, 82, 3],
 [20, 10, 14, 29, 75, 117, 3],
 [20, 11, 9, 26, 57, 135, 6],
 [20, 12, 5, 26, 38, 154, 6],
 [20, 13, 3, 23, 20, 172, 9],
 [20, 14, 2, 20, 13, 179, 12],
 [20, 15, 2, 20, 13, 179, 12],
 [20, 16, 1, 16, 5, 187, 16],
 [20, 17, 0, 0, 0, 192, 32],
 [20, 18, 0, 0, 0, 192, 32],
 [20, 19, 0, 0, 0, 192, 32],
 [20, 20, 0, 0, 0, 192, 32],
 [19, 3, 2162, 32, 192, 0, 0],
 [19, 4, 1039, 32, 192, 0, 0],
 [19, 5, 495, 32, 192, 0, 0],
 [19, 6, 250, 32, 190, 2, 0],
 [19, 7, 122, 32, 180, 12, 0],
 [19, 8, 57, 30, 154, 38, 2],
 [19, 9, 29, 29, 108, 84, 3],
 [19, 10, 13, 28, 69, 123, 4],
 [19, 11, 9, 26, 57, 135, 6],
 [19, 12, 5, 26, 38, 154, 6],
 [19, 13, 3, 23, 20, 172, 9],
 [19, 14, 2, 20, 13, 179, 12],

In [4]:
#output stats of accuracy filter
with open("./optimizingcimpnegHUGO.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newdatset)
